# Read CSV into a data frame
In this step we read the CSV into a data frame and do some basic cleanup steps. 




In [3]:
%%configure -f
{"executorMemory": "58g", "executorCores": 30, "numExecutors":4}


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1564678375152_0001,pyspark,idle,Link,Link,


In [4]:

#spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import lit

sc.setLogLevel("WARN")

#Read a file and then write it to the SQL table
datafile = "/user/hive/warehouse/source/source_new"
#datafile = "/user/hive/warehouse/sourcesue2"
df = spark.read.format('parquet').load(datafile)
#df.insert(len(df.columns), 'batch_id', 0, allow_duplicates=True)

#this should work to add a column to the df 
df = df.withColumn('batch_id', lit(0))
df.show(1)
df.printSchema()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1564678375152_0002,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


+----------------+-------+------------------+------------------+--------------+------+-----------------+-----------------+------------------+------------------+-------------------+-------------------+-------------------+----------------+----------------------------+-------------------------------+-------------------------+----------------------------+------------------+---------------------+-----------------+-----------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+----------------------+-----------------------+----------------------+-----------------------+------------------+------------------+-------------------+-----------------+--------------------+--------------------+------------------+--------------------+------------------+----------------+---------------------+---------------------+------------------------+-------------------------+------------------------+-------------------------+--------------

# (PART 2) Write and READ to Data Pool external Tables in Big Data Cluster
- Write dataframe to SQL external table in Data Pools in Big Data Cluste
- Read SQL external Table to Spark dataframe

In [5]:
import datetime

before = datetime.datetime.now()

#Write from Spark to SQL table using MSSQL Spark Connector
print("Use MSSQL connector to write to master SQL instance ")

servername = "jdbc:sqlserver://master-0.master-svc"
dbname = "LSST"
url = servername + ";" + "databaseName=" + dbname + ";"

dbtable = "SourceDP"
user = "sa"
password = "fooRiuzg54" # Please specify password here

datapool_table = "SourceDP"
datasource_name = "SqlDataPool"

batchsize = 1000000


try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", datapool_table) \
    .option("user", user) \
    .option("password", password) \
    .option("dataPoolDataSource",datasource_name)\
    .option("batchsize",batchsize)\
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) to data pool external table succeeded")

after = datetime.datetime.now()
print (after - before )


An error occurred while calling o81.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 14563 in stage 2.0 failed 4 times, most recent failure: Lost task 14563.3 in stage 2.0 (TID 20906, storage-0-1.storage-0-svc.filedb.svc.cluster.local, executor 3): com.microsoft.sqlserver.jdbc.SQLServerException: The TCP/IP connection to the host data-0-3.data-0-svc, port 1433 has failed. Error: "Connection refused (Connection refused). Verify the connection properties. Make sure that an instance of SQL Server is running on the host and accepting TCP/IP connections at the port. Make sure that TCP connections to the port are not blocked by a firewall.".
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDriverError(SQLServerException.java:226)
	at com.microsoft.sqlserver.jdbc.SQLServerException.ConvertConnectExceptionToSQLServerException(SQLServerException.java:277)
	at com.microsoft.sqlserver.jdbc.SocketFinder.findSocket(IOBuffer.java:2379)
	at com.microsoft.sqls

In [8]:

#testing on small batch of parquet files
#Write from Spark to SQL table using MSSQL Spark Connector
print("Use MSSQL connector to write to master SQL instance ")

servername = "jdbc:sqlserver://master-0.master-svc"
dbname = "LSST"
url = servername + ";" + "databaseName=" + dbname + ";"

dbtable= "Source_test"
#dbtable = "SourceDP"
user = "sa"
password = "fooRiuzg54" # Please specify password here

datapool_table = "Source_test"
datasource_name = "SqlDataPool"

batchsize = 1000000


try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", datapool_table) \
    .option("user", user) \
    .option("password", password) \
    .option("dataPoolDataSource",datasource_name)\
    .option("batchsize",batchsize)\
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) to data pool external table succeeded")